In [1]:
from boknis_eck import BEDatabaseWizard
from spectrum import SfgAverager, DummyPlotter 

from datetime import datetime, date, timedelta
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.dates import MonthLocator, DateFormatter
from matplotlib.lines import Line2D

%matplotlib qt
plt.style.use("qt.mpltstyle")

bz = BEDatabaseWizard()



In [9]:
dic = bz.get_be_spectra_monthwise(sample_type="bulk")
aug =SfgAverager(dic[6], enforce_scale=True)
%matplotlib qt
plt.style.use("qt.mpltstyle")


class Plotter:
    
    def __init__(self):
        
        # meta
        self.path = os.path.abspath("..\..\ipy_plots")
        
        # plotting context
        self.figure = None
        self.axes = None
        self.title = "No title"
        
        # setup
        self.provide_plot_context()
        
    def provide_plot_context(self):
        self.figure = plt.figure()
        self.axes = self.figure.add_subplot(1,1,1)
    
    def plot_averager(self, averager):
        
        self.plot_spec(averager.average_spectrum, color="red")
        self.axes.set_xlabel(averager.average_spectrum.x_unit)
        self.axes.set_ylabel(averager.average_spectrum.y_unit)
        
        for spectrum in averager.spectra:
            self.plot_spec(spectrum, alpha=0.3, marker=None)
        
        print(len(averager.spectra))
    
    def plot_spec(self, spec, marker="o", alpha=1, color="default"):
        
        if color == "default":
            self.axes.plot(spec.x, spec.y, marker=marker, label=spec.name, alpha=alpha)
        else:
            self.axes.plot(spec.x, spec.y, marker=marker, label=spec.name, alpha=alpha, color=color)


test = Plotter()
test.plot_averager(aug)

C:\Users\lange\Desktop\CharmingSFG\SFG\spectrum.py:97: RuntimeWarning: divide by zero encountered in true_divide
  self.normalized_intensity = self.raw_intensity / (self.vis_intensity * self.ir_intensity)


10


In [20]:
p = bz.wz.session.query(bz.wz.boknis_eck).filter(bz.wz.boknis_eck.depth > 1).all()
        
[i for i in p if i.sampling_date.month == 6]